In [1]:
!pip install nltk
!pip install pickle
!pip install xgboost
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd
new_columns=pd.read_csv('../data/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../data/NEISS_10P_Sample.csv')
# sematic=pd.read_csv('../data/10P_sematic_bert.csv')
embedding=pd.read_csv('../data/gist_embedding_10p.csv')
#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt to /Users/eric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eric/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/eric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,374,375,376,377,378,379,380,381,382,383
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,0.034040,-0.036666,0.008797,0.024984,-0.048995,0.002709,-0.020119,-0.058474,-0.022191,0.038710
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,0.024239,-0.000142,0.007268,-0.015545,-0.040856,-0.042023,-0.050380,0.008760,-0.002266,0.043031
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,0.060739,0.014632,0.022998,-0.055498,-0.040177,0.024382,-0.086975,-0.045614,-0.022894,-0.013169
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,0.027231,-0.016303,-0.013848,-0.046165,-0.034185,0.006101,-0.041465,-0.041030,0.040551,0.013185
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,0.053574,-0.069382,0.023767,0.044406,0.034625,0.037567,-0.024320,0.038775,-0.017813,0.001262


In [3]:
data.columns

Index(['Unnamed: 0', 'CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex',
       'Race', 'Other_Race', 'Hispanic', 'Body_Part', 'Diagnosis',
       ...
       '374', '375', '376', '377', '378', '379', '380', '381', '382', '383'],
      dtype='object', length=416)

In [4]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [5]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

Disposition_recode_2
0    296227
1     32874
Name: count, dtype: int64

In [7]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [8]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [9]:
# data['Narrative']=data['Narrative'].str.upper()
data[['Narrative']]

,Narrative
0,14YOM REPORTS HE FELL 1 WEEK AND COMPLAINS OF ...
1,A 28YOM BENT TO PICK UP CRATE AT HOME TO ED WI...
2,35YOMRIDING ON MOUNTAIN BIKE PRACTICING FELL D...
3,14 MONTH OLD FEMALE ABRASION FOR NOSE AND FORE...
4,4YR M PLAYING WITH TOY KITCHEN APPLIANCE AND ...
...,...
346307,11 YOM HELPING LOAD BICYCLE IN VAN CUT HEAD DX...
346308,47YOF PUSHING DOWN FOOD IN BLENDER MAKING A SM...
346309,2 YO MALE FELL DOWN STEPS DX HEAD INJURY
346310,38YOM WAS RIDING A BIKE AND FELL WRIST INJURY


In [15]:
replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')


data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [10]:
data_core=data.copy() #[core_col]

In [11]:
# please change these hard coded numbers accordingly
data_core=data_core.sample(frac=1,random_state=42).reset_index(drop=True)
data_test=data_core.head(21000).reset_index(drop=True)
data_fit=data_core.tail(191347).reset_index(drop=True)

In [12]:
data_ready = data_core
data_ready_test = data_test
data_ready.columns

Index(['Unnamed: 0', 'CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex',
       'Race', 'Other_Race', 'Hispanic', 'Body_Part', 'Diagnosis',
       ...
       '378', '379', '380', '381', '382', '383', 'Disposition_recode',
       'Disposition_recode_2', 'body_string', 'Narrative_LLM'],
      dtype='object', length=420)

In [17]:
drop_list=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 'Narrative_LLM',
 # 'Processed_Narrative',
 # 'Processed_Narrative_LLM'
          ]

drop_list_test=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 # 'Narrative_LLM'
]
 

In [18]:
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open('X_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic'), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=-1)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open('xgboost_tfidf_LLM_v5.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...


In [41]:
X_test_final.head(5)

,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,-0.688581,-0.935222,0.817527,-0.761139,-0.652828,0.004202,-0.202524,0.670022,0.365266,-0.524021,...,0.044480,-1.460328,-0.969432,-0.424290,0.958737,0.771442,0.605041,1.809550,-0.886592,0.810894
1,-0.127114,1.068401,1.211278,-0.761139,-1.261170,-0.067307,0.401968,-1.171033,-0.941745,1.006470,...,-1.958180,0.542523,0.639321,-1.549186,3.117182,0.274781,1.060470,0.891147,0.856496,0.460138
2,0.779871,-0.935222,0.467527,-0.437733,-0.264824,-2.116730,1.270491,0.623171,0.208979,-0.739689,...,0.010165,-0.933003,1.393133,-0.871157,0.060490,0.984578,0.372143,0.502606,-0.498860,-1.560613
3,-1.012504,1.068401,-1.413726,-0.437733,-0.264824,-0.649481,-0.746886,0.398183,1.166810,-1.296542,...,1.479026,0.052324,0.504815,-0.377094,-0.771888,-0.180674,-0.538727,0.210899,-1.125427,-0.448966
4,-0.796555,-0.935222,-1.194976,2.149511,-0.706985,-0.764863,0.125405,-0.634496,-1.264164,0.714660,...,0.080112,0.356393,-1.069071,0.654946,-0.433049,0.367173,0.967608,0.599836,0.599172,1.196029


In [58]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

array([-1.50122628, -1.45747622, -1.41372615, -1.36997608, -1.32622601,
       -1.28247595, -1.23872588, -1.19497581, -1.15122575,  0.46752674,
        0.5112768 ,  0.55502687,  0.642527  ,  0.68627707,  0.73002714,
        0.77377721,  0.81752727,  1.03627761,  1.08002768,  1.21127788,
        1.25502794,  1.29877801])

In [90]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

0.8180479 0.9804617
0.8434001 0.9866623
0.07610136 0.62177515
0.3582114 0.96472424
0.85777724 0.97853804
0.5630455 0.95304084
0.07579315 0.61503655
0.6504902 0.9727951
0.1762011 0.8535414
0.5228751 0.9777255
